![alttext](logo.png)


# Unidad: Clasificación - Sesión 1


* Para poder realizar esta actividad debes haber revisado la lectura correspondiente a la semana.
* Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook y csv).
* Una vez terminada la actividad, comprime la carpeta y sube el `.zip` a la sección correspondiente.

## Descripción de Actividades

* En esta sesión trabajaremos con un archivo `.csv` donde se registraron las características de una serie de individuos para analizar la propensidad a tener alguna enfermedad coronaria.

* `sbp`: Presión Sanguínea Sistólica.
* `tobacco`: Promedio tabaco consumido por día.
* `ldl`: Lipoproteína de baja densidad.
* `adiposity`: Adiposidad.
* `famhist`: Antecedentes familiares de enfermedades cardiácas. (Binaria)
* `types`: Personalidad tipo A
* `obesity`: Obesidad.
* `alcohol`: Consumo actual de alcohol.
* `age`: edad.
* `chd`: Enfermedad coronaria. (dummy)

## Ejercicio 1: Preparar el ambiente de trabajo

* Importe los módulos básicos para el análisis de datos
* Importe `seaborn`.
* Importe `statsmodels`
* Importe `LogisticRegression`, `train_test_split`, `StandardScaler`, `classification_report` y `roc_curve` de `sklearn`

In [16]:
# Imporatamos la triada clásica 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# seaborn
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

def concise_summary(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],
                       'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.': 'Std.Err.'])
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
    
    print("\nPoint Estimates\n\n", estimates)


## Ejercicio 2

* A continuación se presenta el siguiente modelo a estimar:

![alttext](eq1.gif)

* Para ello ejecute los siguientes pasos:
    1. Recodifique `famhist` a dummy, asignando 1 a la categoría minoritaria.
    - Utilice `smf.logit` para estimar el modelo.

* Implemente una función `inverse_logit` que realize el mapeo de log-odds a probabilidad.

* Con el modelo estimado, responda lo siguiente:
    1. ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria?
    - ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    - ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

* Replique el modelo con `smf.ols` y comente las similitudes entre los coeficientes estimados. 
    - _tip_: Utilice ![](https://latex.codecogs.com/svg.latex?\beta/4) 

df = pd.read_csv('southafricanheart.csv')
df = df.drop("Unnamed: 0", axis =1)

In [101]:
df.describe()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,0.415584,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,0.493357,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,0.000000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,0.000000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,0.000000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,1.000000,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,1.000000,78.000000,46.580000,147.190000,64.000000,1.000000


In [11]:
df['famhist'].value_counts()

Absent     270
Present    192
Name: famhist, dtype: int64

In [12]:
df['famhist'] = np.where(df['famhist']=='Present',1,0)

In [17]:
m1_logit = smf.logit('chd ~ famhist', df).fit()
concise_summary(m1_logit)

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5

Goodness of Fit statistics
         Statistics       Value
2             BIC:    574.1655
3  Log-Likelihood:     -280.95
4         LL-Null:     -298.05
5     LLR p-value:  4.9371e-09
6           Scale:      1.0000
7                             

Point Estimates

               Coef.  Std.Err.
Intercept -1.168993  0.143106
famhist    1.168993  0.203255


In [25]:
def invlogit(x):
    return 1 / (1+np.(exp(-x)))

In [36]:
print('la Probabilidad de que alguien que no tenga antecedentes familiares tenga una enfermedad cardiaca es de :',round(invlogit(-1.168993),3))

la Probabilidad de que alguien que no tenga antecedentes familiares tenga una enfermedad cardiaca es de : 0.237


In [37]:
print('la Probabilidad de que alguien que tenga antecedentes familiares tenga una enfermedad cardiaca es de :',round(invlogit(0),3))

la Probabilidad de que alguien que tenga antecedentes familiares tenga una enfermedad cardiaca es de : 0.5


In [38]:
print('La diferencia entre ambos es de', (round(invlogit(0),3))- (round(invlogit(-1.168993),3)))

La diferencia entre ambos es de 0.263


In [34]:
m1_ols = smf.ols('chd ~ famhist', df).fit()

In [35]:
concise_summary(m1_ols)


Goodness of Fit statistics
             Statistics     Value
2                 BIC:  601.4437
3      Log-Likelihood:   -294.59
4         F-statistic:     36.86
5  Prob (F-statistic):  2.66e-09
6               Scale:   0.21050

Point Estimates

               Coef.  Std.Err.
Intercept  0.237037  0.027922
famhist    0.262963  0.043313


In [40]:
1.169/4

0.29225

Los coeficientes son similares, el log odds/4 es 0.29, similar al 0.25 dado por el modelo ols


## Ejercicio 3: Estimación completa

* Implemente un modelo con la siguiente forma

![alttext](eq2.gif)

* Depure el modelo manteniendo las variables con significancia estadística al 95%.
* Compare los estadísticos de bondad de ajuste entre ambos.
* Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.


In [42]:
df.columns 

Index(['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity',
       'alcohol', 'age', 'chd'],
      dtype='object')

In [44]:
m2_logit = smf.logit('chd ~ sbp + tobacco + ldl + adiposity + famhist + typea + obesity + alcohol + age', df).fit()
m2_logit.summary()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Tue, 24 Jul 2018   Pseudo R-squ.:                  0.2080
Time:                        20:40:03   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.055e-22
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.1507      1.308     -4.701      0.000      -8.715      -3.587
sbp            0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco        0.0794      0.027      2.984      0.003       0.027       0.132
ldl            0.1739      0.060      2.915      0.004       0.057       0.291
adiposity      0.0186      0.029      0.635      0.526      -0.039       0.076
famhist        0.9254      0.228      4.061      0.000       0.479       1.372
typea          0.0396      0.012      3.214      0.001       0.015       0.064
obesity       -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol        0.0001      0.004      0.027      0.978      -0.009       0.009
age            0.0452      0.012      3.728      0.000       0.021       0.069
==============================================================================
"""

In [46]:
m3_logit = smf.logit('chd ~  tobacco + ldl +  famhist + typea + age', df).fit()
m3_logit.summary()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Tue, 24 Jul 2018   Pseudo R-squ.:                  0.2020
Time:                        20:54:15   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
                                        LLR p-value:                 2.554e-24
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.4464      0.921     -7.000      0.000      -8.251      -4.642
tobacco        0.0804      0.026      3.106      0.002       0.030       0.131
ldl            0.1620      0.055      2.947      0.003       0.054       0.270
famhist        0.9082      0.226      4.023      0.000       0.466       1.351
typea          0.0371      0.012      3.051      0.002       0.013       0.061
age            0.0505      0.010      4.944      0.000       0.030       0.070
==============================================================================
"""

- Comparación: Segun el Log-Likelihood, el mejor model es el que contiene todas las variables
- La variable que tiene una mayor influencia es el antecedente familiar, seguido, pero muy de lejos, por el de lidoproteína de baja densidad 

In [86]:
averages = pd.DataFrame(df.mean())
averages

,0
sbp,138.326840
tobacco,3.635649
ldl,4.740325
adiposity,25.406732
famhist,0.415584
typea,53.103896
obesity,26.044113
alcohol,17.044394
age,42.816017
chd,0.346320


## Ejercicio 4: Estimación de perfiles

* A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas a probabilidades con `inverse_logit`. Los perfiles a estimar son los siguientes:

1. La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
- La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, __manteniendo todas las demás características constantes__.
- La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, __manteniendo todas las demás características constantes__.


In [131]:
estimates = pd.DataFrame(m2_logit.summary2().tables[1].loc[:, 'Coef.'])

estimates = estimates.drop(['Intercept'])

averages.columns= ['promedio']

estimates['promedio']= averages['promedio']

estimates

,Coef.,promedio
sbp,0.006504,138.326840
tobacco,0.079376,3.635649
ldl,0.173924,4.740325
adiposity,0.018587,25.406732
famhist,0.925370,0.415584
typea,0.039595,53.103896
obesity,-0.062910,26.044113
alcohol,0.000122,17.044394
age,0.045225,42.816017


In [96]:
estimates['valores']  = estimates['Coef.']*estimates['promedio']

In [98]:
sum(estimates['valores'])+ -6.4464

-1.1742243306571964

In [99]:
invlogit(sum(estimates['valores'])+ -6.4464)

0.23609226563063496

La probabilidad de una enfermedad coronaria para la media es es de 0,24 

In [137]:
alto_ldl = estimates
alto_ldl.loc['ldl']['promedio'] = 15.33
alto_ldl['valores']  = alto_ldl['Coef.']*alto_ldl['promedio']

In [134]:
print('La Probabilidad para alguien con alto ldl es',invlogit(sum(alto_ldl['valores'])+ -6.4464))

La Probabilidad para alguien con alto ldl es 0.6609595635623082


In [135]:
bajo_ldl= estimates

bajo_ldl.loc['ldl']['promedio'] = 0.98
bajo_ldl['valores']  = bajo_ldl['Coef.']*bajo_ldl['promedio']

In [136]:
print('La Probabilidad para alguien con bajo ldl es',invlogit(sum(bajo_ldl['valores'])+ -6.4464))

La Probabilidad para alguien con bajo ldl es 0.13844871016752272
